## Imports

In [1]:
import tensorflow as tf
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np

/home/parallels/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## produce category-index from 'labels.txt'

In [2]:
# produce the new category-index
f = open('./labels.txt')

out = {}
for i in f.readlines():
    lns = i.strip('\n')
    for ln in lns.split(','):
        e = ln.strip().split(':')
        out[int(e[0])] = {'id': int(e[0]), 'name': e[1]}

f.close()

out

{0: {'id': 0, 'name': 'Jeep-北京JEEP'},
 1: {'id': 1, 'name': 'Jeep-大切诺基'},
 2: {'id': 2, 'name': 'Jeep-大切诺基SRT'},
 3: {'id': 3, 'name': 'Jeep-指南者'},
 4: {'id': 4, 'name': 'Jeep-牧马人'},
 5: {'id': 5, 'name': 'Jeep-自由侠'},
 6: {'id': 6, 'name': 'MINI'},
 7: {'id': 7, 'name': 'MINI CLUBMAN'},
 8: {'id': 8, 'name': 'MINI-JCW'},
 9: {'id': 9, 'name': 'MINI-JCWPACEMAN'},
 10: {'id': 10, 'name': 'MINI-ROCKETMAN'},
 11: {'id': 11, 'name': 'smart-forfour'},
 12: {'id': 12, 'name': 'smart-forjeremy'},
 13: {'id': 13, 'name': 'smart-roadster'},
 14: {'id': 14, 'name': '三菱-Colt'},
 15: {'id': 15, 'name': '三菱-GC-PHEV'},
 16: {'id': 16, 'name': '三菱-L200'},
 17: {'id': 17, 'name': '三菱-LANCER'},
 18: {'id': 18, 'name': '三菱-Mirage'},
 19: {'id': 19, 'name': '三菱-XR-PHEV'},
 20: {'id': 20, 'name': '三菱-劲炫ASX'},
 21: {'id': 21, 'name': '三菱-君阁'},
 22: {'id': 22, 'name': '三菱-帕杰罗'},
 23: {'id': 23, 'name': '三菱-戈蓝'},
 24: {'id': 24, 'name': '三菱-格蓝迪'},
 25: {'id': 25, 'name': '三菱-欧蓝德'},
 26: {'id': 26, 'name': '三菱

## crop selected part from bounding box

In [3]:
# load image into numpy array
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8), im_height, im_width

In [4]:
# load image into numpy array
image = Image.open('./object_detection/test1.jpg')
(image_np, im_height, im_width) = load_image_into_numpy_array(image)

In [5]:
# x1
x1 = int(0.21775927*1200)

In [6]:
# y1
y1 = int(0.6717261*650)

In [7]:
# x2
x2 = int(0.46072263*1200)

In [8]:
# y2
y2 = int(0.94666123*650)

In [9]:
# define the crop op
image_crop = tf.image.crop_to_bounding_box(
        image_np, 
        offset_height=y1, offset_width=x1, 
        target_height=y2-y1, target_width=x2-x1)

img_crop = Image.fromarray(tf.Session().run(image_crop), 'RGB')

In [10]:
# save crop part to file
img_crop.save('crop.jpg')

## draw the bounding box and write classname on the primary image

In [11]:
# 349: class index
out[349]['name']

'广汽传祺-传祺GS3'

In [12]:
# get the referred classname
car_name = out[349]['name']
len_background = len(car_name.encode('gb2312'))

In [13]:
# read the primary image and initialize the drawing
im = cv2.imread('./object_detection/test1.jpg', cv2.COLOR_BGR2RGB)
pil_im = Image.fromarray(im)
draw = ImageDraw.Draw(pil_im)
font = ImageFont.truetype('./simhei.ttf', 18, encoding='utf-8')

In [14]:
# plot the text background
draw.rectangle((x1, y1, x1 + len_background*9, y1+18), 
               fill=(211, 211, 211), outline=None)
# write classname on the image
draw.text((x1, y1), car_name, (255, 0, 255), font=font)
# plot the bounding box
draw.line((261, 436, 552, 436), (0, 255, 0), width=3)
draw.line((552, 436, 552, 615), (0, 255, 0), width=3)
draw.line((552, 615, 261, 615), (0, 255, 0), width=3)
draw.line((261, 615, 261, 436), (0, 255, 0), width=3)

In [15]:
# save to out1.jpg
im = np.array(pil_im)
cv2.imwrite('./out1.jpg', im)

True